In [4]:
%load_ext autoreload
%autoreload 2
from pydarknet import Libdarknet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import ctypes
import os

In [6]:
self = Libdarknet()

In [7]:
self

In [10]:
self.exists

True

In [11]:
self.lib

<CDLL '/mnt/ubuntu1604_2/home/jed/.darknet/libdarknet.so', handle 2079380 at 0x7fea6c908748>

In [12]:
lib = self.lib

In [79]:
class Detections(object):
    def __init__(self, num, detections_ptr):
        self.num = num
        self.detections_ptr = detections_ptr

    def __repr__(self):
        return "Detections<{}>".format(self.num)

    def __iter__(self):
        self._idx = 0
        return self

    def __next__(self):
        idx = self._idx
        if self._idx >= self.num:
            raise StopIteration
        else:
            self._idx += 1
            return self.detections_ptr[idx]

    def __getitem__(self, index):
        return self.detections_ptr[index]

In [80]:
# Relative files are **relative** to the darknet directory!
metadata = self.get_metadata("cfg/coco.data")
network = self.load_network("cfg/yolov3.cfg", "weights/yolov3.weights", 0)
img = self.load_image_color("data/dog.jpg")

In [81]:
net_output = self.network_predict_image(network, img)

In [82]:
net_output

In [83]:
num, detections_ptr = self.get_network_boxes(network, img)
dets = Detections(num, detections_ptr)
for det in dets:
    print(det)

Detection<Box<561.5, 121.6, 214.3, 86.5>, 0.79>
Detection<Box<580.8, 125.1, 207.9, 87.1>, 1.00>
Detection<Box<342.8, 260.5, 470.4, 320.6>, 0.89>
Detection<Box<382.5, 259.5, 422.5, 324.4>, 0.62>
Detection<Box<317.7, 290.0, 450.0, 329.5>, 0.56>
Detection<Box<344.4, 285.9, 490.3, 323.4>, 0.99>
Detection<Box<376.8, 284.4, 431.9, 322.8>, 0.91>
Detection<Box<416.0, 282.7, 301.3, 277.3>, 0.67>
Detection<Box<411.7, 281.2, 364.8, 316.6>, 0.54>
Detection<Box<219.3, 332.9, 178.0, 337.8>, 0.69>
Detection<Box<347.2, 318.8, 464.9, 340.2>, 0.80>
Detection<Box<224.3, 378.4, 178.8, 328.3>, 1.00>
Detection<Box<222.7, 392.9, 177.9, 321.1>, 1.00>


In [84]:
self.do_nms_sort(detections_ptr, num, metadata.classes, nms);
dets = Detections(num, detections_ptr)
for det in dets:
    print(det)

Detection<Box<224.3, 378.4, 178.8, 328.3>, 1.00>
Detection<Box<222.7, 392.9, 177.9, 321.1>, 1.00>
Detection<Box<219.3, 332.9, 178.0, 337.8>, 0.69>
Detection<Box<580.8, 125.1, 207.9, 87.1>, 1.00>
Detection<Box<561.5, 121.6, 214.3, 86.5>, 0.79>
Detection<Box<344.4, 285.9, 490.3, 323.4>, 0.99>
Detection<Box<376.8, 284.4, 431.9, 322.8>, 0.91>
Detection<Box<342.8, 260.5, 470.4, 320.6>, 0.89>
Detection<Box<347.2, 318.8, 464.9, 340.2>, 0.80>
Detection<Box<382.5, 259.5, 422.5, 324.4>, 0.62>
Detection<Box<416.0, 282.7, 301.3, 277.3>, 0.67>
Detection<Box<317.7, 290.0, 450.0, 329.5>, 0.56>
Detection<Box<411.7, 281.2, 364.8, 316.6>, 0.54>


In [94]:
res = []
for j in range(num):
    for i in range(metadata.classes):
        if dets[j].prob[i] > 0:
            b = dets[j].bbox
            res.append((metadata.names[i], dets[j].prob[i], (b.x, b.y, b.w, b.h)))

In [107]:
for det in dets:
    break

In [111]:
assert det.classes == metadata.classes

In [112]:
for i in range(det.classes):
    break

In [119]:
res = []
for det in dets:
    assert det.classes == metadata.classes
    for i in range(metadata.classes):
        if det.prob[i] > 0:
            b = det.bbox
            res.append((metadata.names[i], det.prob[i], (b.x, b.y, b.w, b.h)))
res

[(b'dog',
  0.9993110299110413,
  (224.25091552734375,
   378.3516845703125,
   178.82186889648438,
   328.3298034667969)),
 (b'truck',
  0.9163237810134888,
  (580.8344116210938,
   125.05101013183594,
   207.87521362304688,
   87.08351135253906)),
 (b'bicycle',
  0.9936886429786682,
  (344.4174499511719,
   285.91204833984375,
   490.32781982421875,
   323.41693115234375))]